In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

In [ ]:
# LLM interface
from langchain_google_genai.llms import GoogleGenerativeAI

model = GoogleGenerativeAI(model="gemini-2.0-flash")
model.invoke("The grass is")

In [ ]:
# Chat interface
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt = [HumanMessage(content="What is the capital of Germany?")]
model.invoke(prompt)

In [ ]:
system_msg = SystemMessage(
    "You are a helpful assistant who responds to questions using bold text."
)
human_msg = HumanMessage("What is the capital of Germany?")
prompt = [system_msg, human_msg]
model.invoke(prompt)